In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = "datasets/paddy-images/train_images"
valid_dir = "datasets/paddy-images/valid_images"

In [3]:
image_size = (128, 128)
batch_size = 32

# ✅ Use ONLY ONE `ImageDataGenerator` for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,       # Rotate images by 20 degrees
    width_shift_range=0.2,   # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest'      # Fill in missing pixels
)

# ✅ Validation data should NOT be augmented
valid_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

Found 8384 images belonging to 10 classes.
Found 2023 images belonging to 10 classes.


In [4]:
from tensorflow.keras import Input

In [5]:
model = Sequential([
    Input(shape=(128, 128, 3)),  # Explicitly define input shape
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Dynamic class count
])

# Compile the model
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,930 (12.61 MB)

 Trainable params: 3,305,930 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
epochs = 50  # Increase if needed

history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs
)

/Users/yash/.pyenv/versions/3.11.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.1829 - loss: 2.2232

/Users/yash/.pyenv/versions/3.11.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


262/262 ━━━━━━━━━━━━━━━━━━━━ 49s 184ms/step - accuracy: 0.1830 - loss: 2.2229 - val_accuracy: 0.2832 - val_loss: 2.0081
Epoch 2/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 53s 203ms/step - accuracy: 0.2886 - loss: 2.0017 - val_accuracy: 0.3485 - val_loss: 1.8132
Epoch 3/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 54s 207ms/step - accuracy: 0.3313 - loss: 1.8946 - val_accuracy: 0.3786 - val_loss: 1.7556
Epoch 4/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 58s 222ms/step - accuracy: 0.3549 - loss: 1.8347 - val_accuracy: 0.3520 - val_loss: 1.8356
Epoch 5/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 57s 219ms/step - accuracy: 0.3760 - loss: 1.7869 - val_accuracy: 0.4296 - val_loss: 1.6659
Epoch 6/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 56s 213ms/step - accuracy: 0.4051 - loss: 1.7372 - val_accuracy: 0.4399 - val_loss: 1.6066
Epoch 7/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 56s 215ms/step - accuracy: 0.4083 - loss: 1.6971 - val_accuracy: 0.4474 - val_loss: 1.5690
Epoch 8/50
262/262 ━━━━━━━━━━━━━━━━━━━━ 57s 217ms/step - accuracy: 0.4478 - loss: 1.6077 - val

In [7]:
model.save("paddy_disease_cnn.h5")
print("Model saved successfully!")

Model saved successfully!
